## Test1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
img = cv2.imread("C:/Users/ferna/OneDrive/Documentos/Insper/Efootball/data/frames/140.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(gray, cmap='gray')
plt.show()

In [ ]:
# Apply thresholding to segment the image
threshold, binary = cv2.threshold(gray, 130, 255, cv2.THRESH_BINARY)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(binary, cmap='gray')
plt.show()

In [ ]:
# Perform morphological operations
kernel = np.ones((3,3),np.uint8)
binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(binary, cmap='gray')
plt.show()

In [ ]:
# Apply contour detection
contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
test = img.copy()
x=[]
for i in range(0, len(contours)):
    if cv2.contourArea(contours[i]) > 100:
        x.append(contours[i])
test = cv2.drawContours(test, x, -1, (255,0,0), 2) 

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(test, cmap='gray')
plt.show()

In [ ]:
# Create a mask to remove the identified objects
mask = np.zeros_like(img)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(mask, cmap='gray')
plt.show()

In [ ]:
for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    mask[y:y+h, x:x+w, :] = img[y:y+h, x:x+w, :]

# Apply the mask to the original image
result = cv2.bitwise_and(img, mask)

# Save the result
plt.figure(figsize=(10,20))
plt.imshow(result, cmap='gray')
plt.show()


## Hough lines

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
field = cv2.imread("C:/Users/ferna/OneDrive/Documentos/Insper/Efootball/data/frames/630.png")
field_gray= cv2.cvtColor(field,cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(10,20))
plt.imshow(field_gray, cmap='gray')
plt.show()

In [ ]:
edges = cv2.Canny(field_gray, 50, 150)

In [ ]:
edges = edges/255

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(edges, cmap='gray')
plt.show()

In [ ]:
kernel = np.ones((5,5),np.uint8)
dilation = cv2.dilate(edges, kernel, iterations = 1)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(dilation, cmap='gray')
plt.show()

In [ ]:
dilation = dilation.astype(np.uint8)
mask_blurred = cv2.medianBlur(dilation, 9)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(mask_blurred, cmap='gray')
plt.show()

In [ ]:
frame = field.copy()
lines = cv2.HoughLinesP(mask_blurred, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=20)
if lines is not None:
    print(len(lines))
    for line in lines:
        x1,y1,x2,y2 = line[0]
        length = round(math.sqrt((x2 - x1)**2 + (y2 - y1)**2), 1)
        angle = round(math.degrees(math.atan2(y2 - y1, x2 - x1)), 1)
        
        if (abs(angle) > 10 and abs(angle) < 30)  or (abs(angle) > 60 and abs(angle) < 130):
            frame = cv2.line(frame,(x1,y1),(x2,y2),(255,0,0),2)
            cv2.putText(frame, str(angle), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            #cv2.putText(frame, str(int(length)), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

In [ ]:
plt.figure(figsize=(20,40))
plt.imshow(frame)
plt.show()

In [ ]:
#remove the dilation from the edges
intersection = dilation*edges
mask_to_keep = edges-intersection

In [ ]:
#apply the dilation mask on the field image
#convert field to RGB
field_rgb = cv2.cvtColor(field, cv2.COLOR_BGR2RGB)
field_masked = cv2.bitwise_and(field_rgb, field_rgb, mask=mask_blurred)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(field_masked)
plt.show()

In [ ]:
def get_lines(image):
    field_gray= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(field_gray, 50, 150)
    edges = edges/255
    kernel = np.ones((5,5),np.uint8)
    dilation = cv2.dilate(edges, kernel, iterations = 1)
    dilation = dilation.astype(np.uint8)
    mask_blurred = cv2.medianBlur(dilation, 9)
    frame = image.copy()
    lines = cv2.HoughLinesP(mask_blurred, rho=1, theta=np.pi/180, threshold=150, minLineLength=530, maxLineGap=1)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line[0]
            length = round(math.sqrt((x2 - x1)**2 + (y2 - y1)**2), 1)
            angle = round(math.degrees(math.atan2(y2 - y1, x2 - x1)), 1)
            if angle > 20:
                frame = cv2.line(frame,(x1,y1),(x2,y2),(255,0,0),2)
                cv2.putText(frame, str(angle), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    return frame

In [ ]:
# import some common libraries
import numpy as np
import cv2
import random
import os
import matplotlib.pyplot as plt

from efootball.src.classes.models.detectron2_field.ModelField import FieldDetector
from efootball.src.classes.models.detectron2_person.ModelPerson import PersonDetector 

In [ ]:
person_detector = PersonDetector(0.5)
field_detector = FieldDetector(0.5)

In [ ]:
def get_only_field_lines(image):
    field = field_detector.removeBackground(image)
    persons = person_detector.detectPersons(field)
    gray3d= cv2.cvtColor(field,cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray3d, 50, 150)
    edges = edges/255
    person_mask = np.zeros_like(edges)
    for i in range(len(persons['masks'])):
        temp_mask = persons['masks'][i].cpu().numpy()
        person_mask[temp_mask] = 1
    kernel = np.ones((4,4),np.uint8)
    dilated_person_mask = cv2.dilate(person_mask, kernel, iterations = 2).astype(np.uint8)
    inverted_person_mask = 1 - dilated_person_mask
    mask_a = edges.squeeze().astype(np.uint8)
    mask_b = inverted_person_mask.squeeze().astype(np.uint8)
    image_without_persons = np.bitwise_and(mask_b, mask_a)
    kernel = np.ones((3,3),np.uint8)
    dilation = cv2.dilate(image_without_persons, kernel, iterations = 3).astype(np.uint8)
    kernel = np.ones((2,8),np.uint8)
    erosion = cv2.erode(dilation ,kernel,iterations = 1)
    erosion = erosion.astype(np.uint8)
    mask_blurred = cv2.medianBlur(erosion, 5)
    field_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    field_masked = cv2.bitwise_and(field_rgb, field_rgb, mask=mask_blurred)
    return field_masked


## Apply to video

In [ ]:
import tqdm
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

In [17]:
VIDEO_PATH = r'C:/Users/ferna/OneDrive/Documentos/Insper/Efootball/data/input/08fd33_0.mp4'
OUT_VIDEO = r'C:/Users/ferna/OneDrive/Documentos/Insper/Efootball/data/output/08fd33_0_out.mp4'
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out_cap = cv2.VideoWriter(OUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

In [18]:
pbar = tqdm.tqdm(total=n_frames)
for i in range(500):
    ret, frame = cap.read()
    frame = get_only_field_lines(frame)
    out_cap.write(frame)
    pbar.update(1)
pbar.close()
cap.release()
out_cap.release()

 67%|██████▋   | 500/750 [04:04<02:02,  2.04it/s]
